In [ ]:
import time
import numpy as np
from PIL import Image

In [ ]:
imarr=np.array(Image.open('lena.gif').convert('L'))

In [ ]:
lookup = np.array([
            [0, 7, 13, 10],
            [1, 6, 12, 11],
            [4, 2, 9, 15],
            [5, 3, 8, 14]], dtype=np.uint8)

In [ ]:
from embedding import watermark_embedding
embedded=watermark_embedding(imarr, lookup).imarr
embedded.shape

In [ ]:
Image.fromarray(embedded)

In [ ]:
from utils import psnr
psnr(imarr, embedded)

In [ ]:
# save
Image.fromarray(embedded).save('lena_embedded.png')

In [ ]:
# raise Exception("stop")

temper img

In [ ]:
startx = 200
starty = 200
width = 50
height = 50

for i in range(width):
    for j in range(height):
        embedded[startx+i][starty+j] = 0

In [ ]:
startx = 100
starty = 100
width = 50
height = 50

for i in range(width):
    for j in range(height):
        embedded[startx+i][starty+j] = 0

In [ ]:
imarr=embedded.copy()

Recovrey

In [ ]:
# imarr=np.array(Image.open('lena_embedded_edited.png').convert('L'))

In [ ]:
Image.fromarray(imarr)

In [ ]:
from authentication import watermark_authentication
tempred=watermark_authentication(imarr).tempred
tempred.shape,tempred

In [ ]:
from recoverybits import extract
recovery_bits=extract.extract(embedded)
recovery_bits.shape

In [ ]:
# raise Error

In [ ]:
recovery_bits

In [ ]:
lookup

In [ ]:
partner=0
id=0
np.any(tempred[lookup[partner,id]])

In [ ]:
partner_block = -1
for i in range(lookup[partner].shape[0]):
    # if this id isn't tempred, we can stop
    if i != id and not np.any(tempred[lookup[partner,i]]):
        partner_block = partner,i
        break

In [ ]:
partner_block, (partner,id)

In [ ]:
idx = id
if partner_block[1] < id:
  idx -= 1
idx

In [ ]:


# now we have partner_block, so we can recover it
# get the recovery bits of this partner block
recovery_bits_partner = recovery_bits[lookup[partner_block],
                                  idx*recovery_bits.shape[1]//3:idx+recovery_bits.shape[1]//3]

# now we have recovrey bits the exact partner block
# but these are in binary, so we need to convert them to uint
recovery_decimals = np.packbits(recovery_bits_partner)

In [ ]:
recovery_bits.shape

In [ ]:
recovery_bits_partner.shape

In [ ]:
recovery_decimals.shape

In [ ]:
Image.fromarray(recovery_decimals.reshape(32,32))

In [ ]:
recovery_decimals=recovery_decimals.reshape(16,-1)
recovery_decimals.shape

In [ ]:
Image.fromarray(recovery_decimals)

In [ ]:
Image.fromarray(recovery_decimals.reshape(32,32))

In [ ]:
import math

In [ ]:
newshape=recovery_decimals.shape[0]*recovery_decimals.shape[1]*4*4
newshape, math.sqrt(newshape)

In [ ]:
from blocks import get_blocks, combine_blocks


In [ ]:
# making 16 main blocks
# size of single main block S= M/sqrt(16) X T=N/sqrt(16)
# e.g for lena the 512x512 the partner blocks size would be 16 blocks each with size (512x512)/(4x4) = 128x128
mainblock_shape = (int(imarr.shape[0]/4), int(imarr.shape[1]/4))
mainblocks = get_blocks(imarr.copy(), mainblock_shape)

# here the main blocks are like this index.
# 1st index is the block number.
# 2nd index is the channel (RGB) or 0 in Grey.
# 3rd and 4th are for indexing the block.

# Making 16x16 for Step 8
# Dividing the main blocks to 16x16 blocks
# Total blocks = SxT/16x16 = 128x128/16x16 = 64 Blocks

b16x16_shape = (16, 16)
# reshaping because we needed that shape
b16x16 = mainblocks.reshape(
  *mainblocks.shape[:-2],
  int((mainblock_shape[0]*mainblock_shape[1]) /
      (b16x16_shape[0]*b16x16_shape[1])),
  b16x16_shape[0], b16x16_shape[1]).copy()

for i in range(mainblocks.shape[0]):
  b16x16[i][0] = get_blocks(mainblocks[i][0], b16x16_shape,
                            addChannel=False)

In [ ]:
Image.fromarray(b16x16[0][0][0])

In [ ]:
b16x16.shape

In [ ]:
b16x16[0][0][0].shape

In [ ]:
b4x4.shape

In [ ]:
r4x4=get_blocks(recovery_decimals.reshape(32,32), (4,4))

In [ ]:
recovery_decimals.shape

In [ ]:
import scipy.ndimage

In [ ]:
b16x16.shape

In [ ]:
tempred.shape

In [ ]:
int(imarr.shape[0]/(4*4))

In [ ]:
Image.fromarray(recovery_decimals)

In [ ]:
x=scipy.ndimage.zoom(recovery_decimals.reshape(32,32), 4, order=0)
x.shape

In [ ]:
r4x4=get_blocks(x, (16,16), addChannel=False)
r4x4.shape

In [ ]:
Image.fromarray(r4x4[0])

In [ ]:
r4x4.shape

In [ ]:
r4x4

In [ ]:
Image.fromarray

In [ ]:
start_t = time.time()

recoveredarr = imarr.copy()

for partner in lookup:  # A,B,C,D
    for id in partner:  # A1,A2,A3.....D4 etc
        # check if this is tempred
        if np.any(tempred[id]):
            # this mainblock is tempred, so we need to recover it
            # but which 16x16 block is tempred?

            # get the partner block of this id also the id it sits
            partner_block = -1
            for pb in partner:
                # if this id isn't tempred, we can stop
                if pb != id and not np.any(tempred[pb]):
                    partner_block = pb
                    break

            if partner_block == -1:
                print(
                    "Could not find partner block for id {}".format(id))
                continue

            # get recovery bits of this id
            # get index of this id in the partner block
            idx = partner_block
            if partner_block < id:
                idx -= 1

            # now we have partner_block, so we can recover it
            # get the recovery bits of this partner block
            recovery_bits_partner = recovery_bits[partner_block,
                                                  idx*recovery_bits.shape[1]//3:idx+recovery_bits.shape[1]//3]

            # now we have recovrey bits the exact partner block
            # but these are in binary, so we need to convert them to uint
            recovery_decimals = np.packbits(recovery_bits_partner)

            # these recovery bits are for all the main blocks 4x4
            # but we are going to replace only the tempred 16x16
            

In [ ]:
recovery_decimals

In [ ]:
from typing import NamedTuple
import numpy as np
import scipy.ndimage
import time
from blocks import get_blocks, combine_blocks
from recoverybits import make as recoverybits_make, embed as recoverybits_embed, extract as recoverybits_extract
from authbits import make as authbits_make, embed as authbits_embed, extract as authbits_extract



In [ ]:
imarr.shape, recovery_bits.shape, tempred.shape

In [ ]:

start_t = time.time()

recoveredarr = imarr.copy()

In [ ]:



# only do if any of the blocks are tempred
# if np.any(tempred):

# converting the image to 16x16 blocks
# making 16 main blocks
# size of single main block S= M/sqrt(16) X T=N/sqrt(16)
# e.g for lena the 512x512 the partner blocks size would be 16 blocks each with size (512x512)/(4x4) = 128x128
mainblock_shape = (int(imarr.shape[0]/4), int(imarr.shape[1]/4))
mainblocks = get_blocks(imarr.copy(), mainblock_shape)

# here the main blocks are like this index.
# 1st index is the block number.
# 2nd index is the channel (RGB) or 0 in Grey.
# 3rd and 4th are for indexing the block.

# Making 16x16 for Step 8
# Dividing the main blocks to 16x16 blocks
# Total blocks = SxT/16x16 = 128x128/16x16 = 64 Blocks

b16x16_shape = (16, 16)
# reshaping because we needed that shape
b16x16 = mainblocks.reshape(
    *mainblocks.shape[:-2],
    int((mainblock_shape[0]*mainblock_shape[1]) /
        (b16x16_shape[0]*b16x16_shape[1])),
    b16x16_shape[0], b16x16_shape[1]).copy()

for i in range(mainblocks.shape[0]):
    b16x16[i][0] = get_blocks(mainblocks[i][0], b16x16_shape,
                                addChannel=False)

for partner in range(lookup.shape[0]):  # A,B,C,D
    for id in range(lookup.shape[1]):  # A1,A2,A3.....D4 etc
        # check if this is tempred
        if np.any(tempred[lookup[partner, id]]):
            # this mainblock is tempred, so we need to recover it
            # but which 16x16 block is tempred?

            # get the partner block of this id also the id it sits
            partner_block = -1, -1
            for i in range(lookup[partner].shape[0]):
                # if this id isn't tempred, we can stop
                if i != id and not np.any(tempred[lookup[partner, i]]):
                    partner_block = partner, i
                    break

            if partner_block == (-1, -1):
                print(
                    "Could not find partner block {} for id {}".format(partner,id))
                continue

            # get recovery bits of this id
            # get index of this id in the partner block
            idx = id
            if partner_block[1] < id:
                idx -= 1

            # now we have partner_block, so we can recover it
            # get the recovery bits of this partner block
            recovery_bits_partner = recovery_bits[lookup[partner_block],
                                                    idx*recovery_bits.shape[1]//3:idx*recovery_bits.shape[1]//3+recovery_bits.shape[1]//3]

            # now we have recovrey bits the exact partner block
            # but these are in binary, so we need to convert them to uint
            recovery_decimals = np.packbits(recovery_bits_partner)

            # these recovery bits are for all the main blocks 4x4
            # but we are going to replace only the tempred 16x16

            # shaping these recovery bits to the 16x16 blocks
            r16x16 = get_blocks(scipy.ndimage.zoom(recovery_decimals.reshape(int(
                imarr.shape[0]/(4*4)), int(imarr.shape[1]/(4*4))), 4, order=0), b16x16_shape, addChannel=False)

            # now we repace which ever 16x16 block is tempred
            for i in range(tempred[lookup[partner, id]].shape[0]):
                if tempred[lookup[partner, id], i]:
                    # means this 16x16 block is tempred
                    b16x16[lookup[partner, id], 0, i] = r16x16[i]

# combining the 16x16 blocks to the main blocks
# merging 16x16
cmainblocks = mainblocks.copy()
for i in range(cmainblocks.shape[0]):
    cmainblocks[i][0] = combine_blocks(
        b16x16[i][0], mainblock_shape, blockshape=b16x16_shape, channel=False)

# merging main blocks to main image
recoveredarr = combine_blocks(cmainblocks.copy(), imarr.shape,
                                mainblock_shape).reshape(imarr.shape)



In [ ]:
Image.fromarray(tempred.reshape(32,32))

In [ ]:
recovery_bits[0].shape

In [ ]:
idx*recovery_bits.shape[1]//3,idx*recovery_bits.shape[1]//3+recovery_bits.shape[1]//3

In [ ]:
recovery_bits[lookup[partner_block],
                                                    idx*recovery_bits.shape[1]//3:idx+recovery_bits.shape[1]//3]

In [ ]:
recovery_decimals

In [ ]:
recovery_bits.shape[1]//3

In [ ]:
partner, id, idx, partner_block

In [ ]:
lookup

In [ ]:
idx

In [ ]:
recoveredarr.shape

In [ ]:
Image.fromarray(recoveredarr.astype(np.uint8))